In [1]:
%load_ext autoreload
%autoreload 2

import requests
import msgpack
import zlib
import traceback
import time
import os
import boto3
from concurrent.futures import ThreadPoolExecutor 
import concurrent.futures 
import datetime 
import pandas as pd 
from tqdm.notebook import tqdm
import numpy as np 
import pickle 
from src.model import OrderBookChunksCollection, OrderBook, S3OrderBookDataSource, OrderBooksDataSequenceBase


In [2]:
ALL_KEYS_CACHED_PICKLE_FILE = "all_keys.pkl"
all_keys = pickle.load(open(ALL_KEYS_CACHED_PICKLE_FILE, "rb"))
import random 
random.shuffle(all_keys)

In [3]:
s3_client = boto3.client("s3")
source = S3OrderBookDataSource(bucket_name="btc-order-book", s3_client=s3_client)
order_books_sample = source.get_all_order_books(keys=all_keys[::300])

In [31]:
amounts_usds = []
for order_book in order_books_sample:
    for bid in order_book.bids:
        amounts_usds.append(bid.amount_usd)
    for ask in order_book.asks:
        amounts_usds.append(ask.amount_usd)
    


In [32]:
s = pd.Series(amounts_usds + [0, np.inf])

In [35]:
s = pd.qcut(s, 246, labels=None)

In [38]:
amount_usd_indices = dict(zip(s.cat.categories, range(246)))
amount_best_bins = s.cat.categories

In [39]:
import pickle 
pickle.dump(file=open("data/amount_usd_best_bins.bin", "wb"), obj=amount_best_bins)
pickle.dump(file=open("data/amount_usd_indices.bin", "wb"), obj=amount_usd_indices)


In [23]:
x = PRICE_DIFF_BEST_BINS[-1]

In [28]:
PRICE_DIFF_INDICES

{Interval(-1e-05, 0.00029188, closed='right'): 0,
 Interval(0.00029188, 0.00031823, closed='right'): 1,
 Interval(0.00031823, 0.00033444, closed='right'): 2,
 Interval(0.00033444, 0.00045606, closed='right'): 3,
 Interval(0.00045606, 0.0004743, closed='right'): 4,
 Interval(0.0004743, 0.00058173, closed='right'): 5,
 Interval(0.00058173, 0.00066889, closed='right'): 6,
 Interval(0.00066889, 0.00070334, closed='right'): 7,
 Interval(0.00070334, 0.00071348, closed='right'): 8,
 Interval(0.00071348, 0.0007601, closed='right'): 9,
 Interval(0.0007601, 0.0007682, closed='right'): 10,
 Interval(0.0007682, 0.00082496, closed='right'): 11,
 Interval(0.00082496, 0.0008432, closed='right'): 12,
 Interval(0.0008432, 0.00092225, closed='right'): 13,
 Interval(0.00092225, 0.00093239, closed='right'): 14,
 Interval(0.00093239, 0.00096279, closed='right'): 15,
 Interval(0.00096279, 0.0010337, closed='right'): 16,
 Interval(0.0010337, 0.0010418, closed='right'): 17,
 Interval(0.0010418, 0.0010601, clo

In [24]:
x.right = x.left + 100

AttributeError: attribute 'right' of 'pandas._libs.interval.Interval' objects is not writable